In [40]:
import replicate
import json



In [41]:
with open("dataset/qqa/QQA_test.json", 'r') as file:
    qqa_data = json.loads(file.read())
print(len(qqa_data))

SCI = True
# MODEL = "meta/llama-2-70b-chat"
MODEL = "meta/meta-llama-3-70b-instruct"

162


In [42]:
def load_prompt(dataset):
    """
    Fill the prompt with the dataset
    """
    filled = []
    for d in dataset:
        if SCI:
            question = d["question_sci_10E"]
        else:
            question = d["question"]
        option1 = d["Option1"]
        option2 = d["Option2"]
        prompt = f"""Example 1:
        Question: The ranger and the rustler both were riding horses that galloped at the same speed.  The rustler left at 01:00 where as the ranger left at 0500 hours. Who has traveled further??
        Option 1: the ranger
        Option 2: the rustler
        Answer: Option 2
        Example 2:
        Question: Tina is racing her two dogs. Her greyhound weighs 40 kgs, and her rottweiler weighs 35 kgs. The dog that gets faster more quickly is the? 
        Option 1: rottweiler. 
        Option 2: greyhound
        Answer: Option 1
        ==========
        Answer the following question beginning with ``Answer: Option..``
        Question: {question}
        Option 1: {option1}
        Option 2: {option2}"""
        filled.append(prompt)
    return filled



In [43]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
prompts = load_prompt(qqa_data)

In [46]:
import replicate
from tqdm import tqdm

raw_predictions = []
for prompt in tqdm(prompts, desc="Retrieving response"):
    input = {
    "prompt": prompt,
    "max_new_tokens": 20
    }
    output = replicate.run(
    MODEL,
    input=input
    )
    raw_predictions.append("".join(output))
    # print("".join(output))

Retrieving response: 100%|██████████| 162/162 [02:48<00:00,  1.04s/it]


In [47]:
llama2_predictions = [predict.strip()[8:16] for predict in raw_predictions]
llama2_predictions

['Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',

In [48]:
gold = [d["answer"] for d in qqa_data]
gold

['Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 1',
 'Option 2',
 'Option 2',
 'Option 2',
 'Option 1',
 'Option 2',
 'Option 2',

Now we examine the accuracy of the output:


In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
f1 = f1_score(gold, llama2_predictions, average="micro")
recall = recall_score(gold, llama2_predictions, average="micro")
precision = precision_score(gold, llama2_predictions, average="micro")
res_dict = {
    "model":MODEL,
    "f1":f1,
    "recall":recall,
    "precision":precision,
    "SCI":SCI
}
res_dict

{'model': 'meta/meta-llama-3-70b-instruct',
 'f1': 0.6851851851851852,
 'recall': 0.6851851851851852,
 'precision': 0.6851851851851852,
 'SCI': False}